### This genetic algorithm is an adaptation of [MourvanZhou's evolutionary algorithm code](https://github.com/MorvanZhou/Evolutionary-Algorithm/blob/master/tutorial-contents/Genetic%20Algorithm/Genetic%20Algorithm%20Basic.py) ###

## Squashed entanglement of a state ###

#### Imports

In [ ]:
from qutip import *
from scipy import arcsin, sqrt, pi
import numpy as np
import scipy.sparse as sp
from qutip.qobj import Qobj
%matplotlib
import matplotlib.pyplot as plt
import itertools
import copy

#### Defining some fuctions

In [ ]:
#rhoABE is a tripartite density matrix with the third subsystem being the ancilla (ancilla is bi-partite when I was
#trying the anti-symmetrix state)
def cond_entropy_mutual(rhoABE):  
    AE = rhoABE.ptrace([0, 2, 3]);
    BE = rhoABE.ptrace([1, 2, 3]);
    ABE = rhoABE;
    E = rhoABE.ptrace([2, 3]);
    return entropy_vn(AE,2) + entropy_vn(BE,2) - entropy_vn(ABE,2) - entropy_vn(E,2); 


def rand_herm_mod(N, X, Y, density=1, dims=None):      #this is a modified version of the rand_herm function from QuTip
    if dims:
        _check_dims(dims, N, N)
    # to get appropriate density of output
    # Hermitian operator must convert via:
    herm_density = 2.0 * arcsin(density) / pi

    X_int = sp.rand(N, N, herm_density, format='csr')
    X_int.data = X - 0.5
    Y_int = X_int.copy()
    Y_int.data = 1.0j * Y - (0.5 + 0.5j)
    X_int = X_int + Y_int
    X_int.sort_indices()
    X_int = Qobj(X_int)
    if dims:
        return Qobj((X_int + X_int.dag()) / 2.0, dims=dims, shape=[N, N])
    else:
        return Qobj((X_int + X_int.dag()) / 2.0)
    

def rand_unitary_mod(N, X, Y, density=1, dims=None):    #this is a modified version of the rand_unitary function from QuTip
    #if dims:
    #    _check_dims(dims, N, N)
    U = (-1.0j * rand_herm_mod(N, X, Y, density)).expm()
    U.data.sort_indices()
    if dims:
        return Qobj(U, dims=dims, shape=[N, N])
    else:
        return Qobj(U)


    
def _check_dims(dims, N1, N2):   #this function is taken directly from QuTip
    if len(dims) != 2:
        raise Exception("Qobj dimensions must be list of length 2.")
    if (not isinstance(dims[0], list)) or (not isinstance(dims[1], list)):
        raise TypeError(
            "Qobj dimension components must be lists. i.e. dims=[[N],[N]]")
    if np.prod(dims[0]) != N1 or np.prod(dims[1]) != N2:
        raise ValueError("Qobj dimensions must match matrix shape.")
    if len(dims[0]) != len(dims[1]):
        raise TypeError("Qobj dimension components must have same length.")
        


def purify(rhoAB): #returns a purification of the input state. The returned pure state has dims A_dim, B_dim, A_dim, B_dim
    X = rhoAB.eigenstates()
    PsiABE = 0
    for i in range(len(X[0])):
        PsiABE = PsiABE + np.sqrt(X[0][i]) * tensor(X[1][i], X[1][i])
    return PsiABE.unit()



def Func(Us):
    SE_ = 0.5 * np.array([cond_entropy_mutual(
        (tensor(identity(A_dim),identity(B_dim),U) * rhoABEF * tensor(identity(A_dim),identity(B_dim),U).dag()).ptrace([0,1,2,3])) 
                         for U in Us])
  
    index = np.where(SE_ == SE_.min())[0][0]
    sigmaABE_ = (tensor(identity(A_dim),identity(B_dim),Us[index]) * rhoABEF * tensor(identity(A_dim),identity(B_dim),Us[index]).dag()).ptrace([0,1,2,3])
    
    return sigmaABE_, SE_

    
# find non-zero fitness for selection.
def get_fitness(pred): return 1/pred - 1/np.max(pred)  #fitness is highest when cmi is lowest

# convert binary DNA to decimal and normalize it to a range(0, 1). Modified: takes unitary seeding list. returns a Unitary
def translateDNA(pop):
    Z = pop.dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1)
    return np.array([rand_unitary_mod(N, Z[i,0], Z[i,1], density=1, dims=[[A_dim,B_dim, F_dim], [A_dim,B_dim, F_dim]])  
                     for i in range(POP_SIZE)])  #The unitary has dims [A_dim, B_dim, F_dim] because the purifying system has dims
#[A_dim, B_dim]

# nature selection wrt pop's fitness.
def select(pop, fitness):    
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True, p=fitness/fitness.sum())
    return pop[idx]

# mating process (genes crossover).
def crossover_and_mutate(individual, pop, mutate=False):
    #crossover
    if np.random.rand() < CROSS_RATE:
        i_ = np.random.randint(0, POP_SIZE, size=1)[0]                        # select another individual from pop
        cross_points = np.random.randint(0, 2, size=DNA_SIZE*2*N**2).astype(bool).reshape((2,N**2,DNA_SIZE))# choose crossover points
        individual[cross_points] = pop[i_][cross_points]
        
    #mutate
    if mutate:
        x = np.random.choice([0, 1], size=DNA_SIZE*2*N**2, 
                             p=[1-MUTATION_RATE, MUTATION_RATE]).astype(bool).reshape((2,N**2,DNA_SIZE))
        individual[x] = np.abs(individual[x] - 1)  #flip the bits
        
    return individual


#### The adapted genetic algorithm. 
#### Each individual in the population is a list of two lists X and Y that seed a unitary.

Define the input state in this cell

In [ ]:
#-------The completely anti-symmetric state. see (page 4, Christandl and Winter 2003)
A_dim = 3
B_dim = 3
F_dim = 2

N = A_dim * B_dim * F_dim

ket1 = (tensor(basis(3,1),basis(3,2)) - tensor(basis(3,2),basis(3,1))).unit()
ket2 = (tensor(basis(3,2),basis(3,0)) - tensor(basis(3,0),basis(3,2))).unit()
ket3 = (tensor(basis(3,0),basis(3,1)) - tensor(basis(3,1),basis(3,0))).unit()
rhoAB = (ket1*ket1.dag() + ket2*ket2.dag() + ket3*ket3.dag()).unit()

ket_pure_ABE = purify(rhoAB)

#ket_pure_ABE = (tensor(ket1, ket1) + tensor(ket2, ket2) + tensor(ket3, ket3)).unit()
temp_psi_ABEF = tensor(ket_pure_ABE, basis(F_dim,0))
rhoABEF = ket2dm(temp_psi_ABEF)
#-------------

In [ ]:
DNA_SIZE = 40           # DNA length   # size of each number in the lists X and Y
POP_SIZE = 100           # population size
CROSS_RATE = 0.001         # mating probability (DNA crossover)
MUTATION_RATE = 0.001    # mutation probability
N_GENERATIONS = 1000


pop = np.random.randint(2, size=([POP_SIZE, 2, N**2, DNA_SIZE]))

#-----------------For plotting
plt.ion()
fig, ax = plt.subplots()
gen, cmi_list = [],[]
ax.scatter(gen,cmi_list)
plt.xlim(0,N_GENERATIONS)
#plt.ylim(0.75,2)
plt.plot([i for i in range(N_GENERATIONS)], 
         [0.5*np.log2(3) for j in range(N_GENERATIONS)]) #this plots the upper bound in the paper.
plt.draw()
plt.xlabel("Generations")
plt.ylabel("Squashed entanglement estimate")
#-----------------------------

for _ in range(N_GENERATIONS):
    sigmaABE, F_values = Func(translateDNA(pop))    # compute function value by extracting DNA
    cmi_list.append(np.max(F_values))
    #-------plot
    ax.scatter(_, cmi_list[-1], c='red')
    plt.pause(0.05)
    #-------

    # GA part (evolution)
    fitness = get_fitness(F_values) #FIXED
    pop = select(pop, fitness)
    pop_copy = pop.copy()
    for parent in pop:
        child = crossover_and_mutate(parent, pop_copy, len(cmi_list)>1 and cmi_list[-1]==cmi_list[-2]) 
        parent[:] = child       # parent is replaced by its child

plt.ioff()

In [ ]:
#Upper bound in the paper can be surpassed.
import math
print(min(cmi_list))
print(0.5*np.log2(3))
print(0.5*math.log(3, 2))

### Now trying the Werner states

In [ ]:
#-------Werner states from (Cope et al. 2018)
d = 2
eta = -0.5  #number between -1 and 0
A_dim = d
B_dim = d
F_dim = 2

F = 0
for i in range(d):
    for j in range(d):
        F = F + tensor(basis(d,i), basis(d,j)) * tensor(basis(d,j), basis(d,i)).dag()

        

rhoAB = (1/(d**3 - d) * ( (d-eta)*tensor(identity(d), 
                                       identity(d)) + (d*eta - 1)*F )).unit()


ket_ABE = purify(rhoAB)
temp_psi_ABEF = tensor(ket_ABE, basis(F_dim,0))
rhoABEF = ket2dm(temp_psi_ABEF)

N = A_dim * B_dim * F_dim


#-------------

first_bound_48 = np.log2(d) + (1+eta)/4 * np.log2((1+eta) / (d*(d+1))) + (1-eta)/4 * np.log2((1-eta) / (d*(d-1)))

if d%2==0:
    second_bound_53 = -eta * np.log2((d + 2) / d)
else:
    second_bound_53 = -0.5 * eta * np.log2((d + 3) / (d - 1))

In [13]:
DNA_SIZE = 30           # DNA length   # size of each number in the lists X and Y
POP_SIZE = 100           # population size
CROSS_RATE = 0.001         # mating probability (DNA crossover)
MUTATION_RATE = 0.005    # mutation probability
N_GENERATIONS = 500


pop = np.random.randint(2, size=([POP_SIZE, 2, N**2, DNA_SIZE]))

#-----------------For plotting
plt.ion()
fig, ax = plt.subplots()
gen, cmi_list = [],[]
ax.scatter(gen,cmi_list)
plt.xlim(0,N_GENERATIONS)
plt.ylim(0,1)
plt.plot([i for i in range(N_GENERATIONS)], [first_bound_48 for j in range(N_GENERATIONS)], c='red')
plt.plot([i for i in range(N_GENERATIONS)], [second_bound_53 for j in range(N_GENERATIONS)], c='blue')
plt.draw()
plt.xlabel("Generations")
plt.ylabel("Squashed entanglement estimate")
#-----------------------------

for gen in range(N_GENERATIONS):
    sigmaABE, F_values = Func(translateDNA(pop))   # compute function value by extracting DNA
    cmi_list.append(np.max(F_values))
    #-------plot
    ax.scatter(gen, cmi_list[-1], c='red')
    plt.pause(0.05)
    #-------

    # GA part (evolution)
    fitness = get_fitness(F_values) #FIXED
    pop = select(pop, fitness)
    pop_copy = pop.copy()
    for parent in pop:
        child = crossover_and_mutate(parent, pop_copy, len(cmi_list)>1 and cmi_list[-1]==cmi_list[-2]) 
        parent[:] = child       # parent is replaced by its child

print(sigmaABE)
plt.ioff()

C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in true_divide
C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in less


Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[  2.56677207e-02+0.j          -6.50662943e-03+0.004283j
   -9.94301249e-04+0.0088496j   -2.80946107e-02+0.00841327j
   -7.63030695e-03-0.01259209j   6.03366568e-02+0.01560042j
   -1.87705646e-02+0.01167044j   2.44058050e-02-0.00525407j
   -8.24957946e-03+0.0322137j   -5.21779732e-02-0.00778974j
    5.13713137e-02-0.01999116j  -1.78685418e-02+0.01517739j
    4.50382561e-03-0.00477473j  -5.69498380e-03+0.01193013j
   -1.04553693e-02+0.01708761j   9.56470671e-03-0.014346j  ]
 [ -6.50662943e-03-0.004283j     2.75201419e-03+0.j
    2.24682789e-03-0.00302048j   1.08966086e-02+0.00198405j
   -3.30101542e-03+0.00793344j  -1.07501690e-02-0.01344569j
    1.09097153e-02+0.00063342j  -8.27590433e-04-0.00237214j
    1.11979809e-02-0.00945655j   1.35545825e-02+0.01124797j
   -2.06248311e-02-0.00239627j   4.86298555e-03-0.00146118j
   -1.21124826e-03-0.00064462j   7.11573145e-03-0.00231886j